In [1]:
from pathlib import Path
from scipy import stats

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from IPython.display import display, Markdown

from sklearn.linear_model import LogisticRegression

In [2]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [3]:
app_train = pd.read_csv(DATASET_DIR / "application_train.csv")
pos = pd.read_csv(DATASET_DIR / "POS_CASH_balance.csv")
des = pd.read_csv(DATASET_DIR / "HomeCredit_columns_description.csv", encoding="latin", index_col=0)

In [4]:
pos.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [5]:
des[des["Table"] == "POS_CASH_balance.csv"]

,Table,Row,Description,Special
145,POS_CASH_balance.csv,SK_ID_PREV,"ID of previous credit in Home Credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loans in Home Credit)",NaN
146,POS_CASH_balance.csv,SK_ID_CURR,ID of loan in our sample,NaN
147,POS_CASH_balance.csv,MONTHS_BALANCE,"Month of balance relative to application date (-1 means the information to the freshest monthly snapshot, 0 means the information at application - often it will be the same as -1 as many banks are not updating the information to Credit Bureau regularly )",time only relative to the application
148,POS_CASH_balance.csv,CNT_INSTALMENT,Term of previous credit (can change over time),NaN
149,POS_CASH_balance.csv,CNT_INSTALMENT_FUTURE,Installments left to pay on the previous credit,NaN
150,POS_CASH_balance.csv,NAME_CONTRACT_STATUS,Contract status during the month,NaN
151,POS_CASH_balance.csv,SK_DPD,DPD (days past due) during the month of previous credit,NaN
152,POS_CASH_balance.csv,SK_DPD_DEF,DPD during the month with tolerance (debts with low loan amounts are ignored) of the previous credit,NaN


In [6]:
pos["SK_ID_CURR"].count()

10001358

In [7]:
print(len(app_train["SK_ID_CURR"].unique()))
print(len(pos["SK_ID_CURR"].unique()))

307511
337252


In [8]:
a = pos[pos["SK_ID_CURR"] == 182943]
a[a["SK_ID_PREV"] == 1215673].sort_values("MONTHS_BALANCE")

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
5036357,1215673,182943,-62,9.0,9.0,Active,0,0
3647834,1215673,182943,-61,9.0,8.0,Active,0,0
6394166,1215673,182943,-60,9.0,7.0,Active,0,0
7323353,1215673,182943,-59,9.0,6.0,Active,0,0
9759959,1215673,182943,-58,9.0,5.0,Active,0,0
8675812,1215673,182943,-57,9.0,4.0,Active,0,0
6783264,1215673,182943,-56,9.0,3.0,Active,0,0
7565356,1215673,182943,-55,9.0,2.0,Active,0,0
9067852,1215673,182943,-54,9.0,1.0,Active,0,0
7022964,1215673,182943,-53,9.0,0.0,Completed,0,0


In [9]:
a[a["SK_ID_PREV"] == 1803195].sort_values("MONTHS_BALANCE")

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
2862153,1803195,182943,-34,48.0,48.0,Active,0,0
6477531,1803195,182943,-33,48.0,47.0,Active,0,0
2865318,1803195,182943,-32,48.0,46.0,Active,0,0
0,1803195,182943,-31,48.0,45.0,Active,0,0
4794783,1803195,182943,-30,48.0,44.0,Active,0,0
8125053,1803195,182943,-29,48.0,43.0,Active,0,0
3337839,1803195,182943,-28,48.0,42.0,Active,0,0
5564316,1803195,182943,-27,48.0,41.0,Active,0,0
5628351,1803195,182943,-26,48.0,40.0,Active,0,0
3093748,1803195,182943,-25,48.0,39.0,Active,0,0


In [10]:
pos[pos["SK_DPD"] > 0]["SK_DPD"].sum() / 365

318041.2191780822

In [11]:
pos[pos["SK_ID_PREV"] == 1394818].sort_values("MONTHS_BALANCE")

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
6063051,1394818,177757,-53,24.0,24.0,Active,0,0
9712546,1394818,177757,-52,24.0,23.0,Active,0,0
6323809,1394818,177757,-51,24.0,22.0,Active,0,0
3712113,1394818,177757,-50,24.0,21.0,Active,0,0
4941488,1394818,177757,-49,24.0,20.0,Active,0,0
3791310,1394818,177757,-48,24.0,19.0,Active,0,0
2723156,1394818,177757,-47,24.0,17.0,Active,0,0
2383367,1394818,177757,-46,24.0,17.0,Active,0,0
3154337,1394818,177757,-45,24.0,16.0,Active,0,0
1816979,1394818,177757,-44,24.0,15.0,Active,0,0


In [12]:
result = {
          "header":[],
          "rate":[],
          "des":[]
         }
for key in pos.keys():
    rate = pos[key].isnull().sum() / len(pos[key]) * 100
    if rate > 60:
        result["header"].append(key)
        result["rate"].append(rate)
        result["des"].append(des[des["Row"] == key]["Description"])

result = pd.DataFrame(result)
result

,header,rate,des


# Informationen
* Anzahl vergangene Kredite
* Anzahl aktuelle Kredite
* Summe Tage Zahlungsverzug
* Kreditlaufzeit
* Zahlungsfortschritt (48-15/48)

In [13]:
SK_ID_CURR = pos["SK_ID_CURR"].unique()
SK_ID_PREV = pos["SK_ID_PREV"].unique()

In [14]:
df = pos[["SK_ID_CURR", "SK_ID_PREV", "CNT_INSTALMENT_FUTURE"]].groupby(by=["SK_ID_PREV"]).min()
len(df[df["CNT_INSTALMENT_FUTURE"] > 0]) / len(SK_ID_PREV) * 100

21.839425413184525

In [15]:
len(df)

936325

In [16]:
df[df["SK_ID_CURR"] == 182943]

,SK_ID_CURR,CNT_INSTALMENT_FUTURE
SK_ID_PREV,,
1215673,182943,0.0
1803195,182943,15.0


In [17]:
df.head()

,SK_ID_CURR,CNT_INSTALMENT_FUTURE
SK_ID_PREV,,
1000001,158271,0.0
1000002,101962,0.0
1000003,252457,9.0
1000004,260094,0.0
1000005,176456,0.0


In [18]:
# abgeschlossene Kredite

a = df[df["CNT_INSTALMENT_FUTURE"] == 0]
b = df[df["CNT_INSTALMENT_FUTURE"] > 0]

past = a[["SK_ID_CURR", "CNT_INSTALMENT_FUTURE"]].groupby(by=["SK_ID_CURR"]).count() # in der Vergangenheit abgeschlossene Kredite
fut = b[["SK_ID_CURR", "CNT_INSTALMENT_FUTURE"]].groupby(by=["SK_ID_CURR"]).count() # aktuell noch laufende Kredite
sum_fut = b[["SK_ID_CURR", "CNT_INSTALMENT_FUTURE"]].groupby(by=["SK_ID_CURR"]).sum() # Summe Zahlungshäufigkeit aktueller Kredite

In [19]:
stop

NameError: name 'stop' is not defined

In [21]:
b[b["SK_ID_CURR"] == 177757]

,SK_ID_CURR,CNT_INSTALMENT_FUTURE
SK_ID_PREV,,
1448484,177757,4.0
2225589,177757,1.0


In [ ]:
sum_fut.loc[182943] # Zum Zeitpunkt der Kreditaufnahme muss KN zukünftig noch 15 weitere Kreditzahlungen vornehmen

In [ ]:
dpd = pos[["SK_ID_CURR","SK_DPD"]].groupby(by=["SK_ID_CURR"]).sum()

In [ ]:
dpd.head()

In [ ]:
dpd.loc[177757] # der KN hat in seiner Kredithistorie 7 Tage nicht gezahlt gehabt

In [ ]:
dpd["SK_DPD"].max()

In [ ]:
# Zusammenführung

In [ ]:
past.columns = ["CNT_CREDITS_PAST"]
fut.columns = ["CNT_CREDITS_CURR"]
sum_fut.columns = ["SUM_PAYCOUNTS_MONTHS"]
dpd.columns = ["SUM_DPD"]

In [ ]:
len(SK_ID_CURR)

In [ ]:
df = pd.DataFrame({"SK_ID_CURR": SK_ID_CURR})
df.head()

In [ ]:
target = app_train[["SK_ID_CURR", "TARGET"]]
target = target.set_index("SK_ID_CURR")

In [ ]:
past.head()

In [ ]:
df = df.join(past, on="SK_ID_CURR")
df = df.join(fut, on="SK_ID_CURR")
df = df.join(sum_fut, on="SK_ID_CURR")
df = df.join(dpd, on="SK_ID_CURR")
#df = df.join(target, on="SK_ID_CURR")

In [ ]:
df.head()

In [ ]:
df = df.fillna(0)

In [ ]:
df.head()

In [ ]:
df = df.set_index("SK_ID_CURR")

In [ ]:
df.head()

In [ ]:
df.loc[182943]

In [ ]:
df.loc[177757]

In [ ]:
len(df)

In [ ]:
#df[df["TARGET"] == 1].head()

In [ ]:
df.to_csv(DATASET_DIR / "Datenaufbereitung" / "pos_mets.csv")